In [1]:
import yt
ds = yt.load("snapshot_200.hdf5")
print(ds)


yt : [INFO     ] 2025-09-17 21:10:50,455 ComovingIntegrationOn does not exist, falling back to OmegaLambda
yt : [INFO     ] 2025-09-17 21:10:50,831 Calculating time from 3.448e-01 to be 1.108e+17 seconds
yt : [INFO     ] 2025-09-17 21:10:50,832 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2025-09-17 21:10:50,897 Parameters: current_time              = 1.1075810732534834e+17 s
yt : [INFO     ] 2025-09-17 21:10:50,898 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-09-17 21:10:50,899 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-09-17 21:10:50,899 Parameters: domain_right_edge         = [64000. 64000. 64000.]
yt : [INFO     ] 2025-09-17 21:10:50,900 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-09-17 21:10:50,900 Parameters: current_redshift          = 1.8999965286929705
yt : [INFO     ] 2025-09-17 21:10:50,900 Parameters: omega_lambda              = 0.721
yt : [INFO     ] 2025-09-17 21:10:50,900 Para

snapshot_200


In [2]:
import h5py

f = h5py.File("snapshot_200.hdf5", "r")

# List all groups at the top level
print("Top-level groups:", list(f.keys()))

# Inspect the header attributes
for key, val in f["Header"].attrs.items():
    print(key, val)

# Look at gas particle fields (PartType0)
print("Gas fields:", list(f["PartType0"].keys()))

# Example: check shapes
print("Coordinates shape:", f["PartType0"]["Coordinates"].shape)
print("Velocities shape:", f["PartType0"]["Velocities"].shape)
print("InternalEnergy shape:", f["PartType0"]["InternalEnergy"].shape)


Top-level groups: ['Header', 'PartType0', 'PartType1', 'PartType2', 'PartType4', 'PartType5']
BoxSize 64000.0
Flag_Cooling 1
Flag_DoublePrecision 0
Flag_Feedback 1
Flag_IC_Info 3
Flag_Metals 12
Flag_Sfr 1
Flag_StellarAge 1
HubbleParam 0.7
MassTable [0.         0.00157866 0.         0.         0.         0.        ]
NumFilesPerSnapshot 1
NumPart_ThisFile [4334546 4786616 2333848       0  450921    1149]
NumPart_Total [4334546 4786616 2333848       0  450921    1149]
NumPart_Total_HighWord [0 0 0 0 0 0]
Omega0 0.279
OmegaLambda 0.721
Redshift 1.8999965286929705
Time 0.3448279989668472
Gas fields: ['ArtificialViscosity', 'Coordinates', 'Density', 'ElectronAbundance', 'InternalEnergy', 'Masses', 'Metallicity', 'NeutralHydrogenAbundance', 'ParticleIDs', 'SmoothingLength', 'StarFormationRate', 'Velocities']
Coordinates shape: (4334546, 3)
Velocities shape: (4334546, 3)
InternalEnergy shape: (4334546,)


In [8]:
import yt, numpy as np

ds = yt.load("snapshot_200.hdf5")
N = 128
left, right = ds.domain_left_edge, ds.domain_right_edge
dims = (N, N, N)

# define smoothing for NeutralHydrogenAbundance
field, units = add_volume_weighted_smoothed_field(
    "PartType0",           # particle type
    "Coordinates",         # positions
    "SmoothingLength",     # SPH smoothing length
    "Masses",              # particle mass
    "NeutralHydrogenAbundance", # the field to smooth
    ds.field_info
)

scg = ds.smoothed_covering_grid(level=0, left_edge=left, dims=dims, fields=fields)

rho  = scg[("gas","density")].to("g/cm**3").ndarray_view()
T    = scg[("gas","temperature")].to("K").ndarray_view()
vx   = scg[("gas","velocity_x")].to("cm/s").ndarray_view()
vy   = scg[("gas","velocity_y")].to("cm/s").ndarray_view()
vz   = scg[("gas","velocity_z")].to("cm/s").ndarray_view()
Z    = scg[("gas","metallicity")].ndarray_view()
xHI  = scg[("gas","NeutralHydrogenAbundance")].ndarray_view()

# Hydrogen number density
X   = 0.76
m_p = 1.6726219e-24
n_H = rho * X / m_p
n_HI = n_H * xHI

np.savez("grid_rtfields.npz", n_HI=n_HI, T=T, vx=vx, vy=vy, vz=vz, Z=Z)


yt : [INFO     ] 2025-09-17 21:20:48,090 ComovingIntegrationOn does not exist, falling back to OmegaLambda
yt : [INFO     ] 2025-09-17 21:20:48,491 Calculating time from 3.448e-01 to be 1.108e+17 seconds
yt : [INFO     ] 2025-09-17 21:20:48,496 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2025-09-17 21:20:48,546 Parameters: current_time              = 1.1075810732534834e+17 s
yt : [INFO     ] 2025-09-17 21:20:48,546 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-09-17 21:20:48,547 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-09-17 21:20:48,547 Parameters: domain_right_edge         = [64000. 64000. 64000.]
yt : [INFO     ] 2025-09-17 21:20:48,548 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-09-17 21:20:48,548 Parameters: current_redshift          = 1.8999965286929705
yt : [INFO     ] 2025-09-17 21:20:48,548 Parameters: omega_lambda              = 0.721
yt : [INFO     ] 2025-09-17 21:20:48,548 Para

NameError: name 'add_volume_weighted_smoothed_field' is not defined

In [6]:
"""
gadget_to_grid.py

Usage:
    python gadget_to_grid.py snapshot_200.hdf5 out_grid.npz
"""

import sys
import numpy as np
import yt
from yt.fields.api import add_smoothed_particle_field

if len(sys.argv) != 3:
    print("Usage: python gadget_to_grid.py <snapshot.hdf5> <output.npz>")
    sys.exit(1)

snapfile = sys.argv[1]
outfile  = sys.argv[2]

# 1. Load the snapshot
ds = yt.load(snapfile)

# 2. Define smoothed fields for NeutralHydrogenAbundance and Metallicity
for pname in ["NeutralHydrogenAbundance", "Metallicity"]:
    add_smoothed_particle_field(
        "PartType0",          # particle type
        "Coordinates",        # particle positions
        "SmoothingLength",    # smoothing length
        "Masses",             # particle masses
        pname,                # particle field name
        ds.field_info,
        ptype_alias="gas"     # expose as ("gas", pname)
    )

# 3. Choose grid resolution
N = 64
dims = (N, N, N)
left  = ds.domain_left_edge
right = ds.domain_right_edge

# 4. Build smoothed covering grid
fields = [
    ("gas","density"), 
    ("gas","temperature"),
    ("gas","velocity_x"), ("gas","velocity_y"), ("gas","velocity_z"),
    ("gas","NeutralHydrogenAbundance"),
    ("gas","Metallicity"),
]

scg = ds.smoothed_covering_grid(level=0, left_edge=left, dims=dims, fields=fields)

# 5. Extract arrays
rho = scg[("gas","density")].to("g/cm**3").ndarray_view()
T   = scg[("gas","temperature")].to("K").ndarray_view()
vx  = scg[("gas","velocity_x")].to("cm/s").ndarray_view()
vy  = scg[("gas","velocity_y")].to("cm/s").ndarray_view()
vz  = scg[("gas","velocity_z")].to("cm/s").ndarray_view()
xHI = scg[("gas","NeutralHydrogenAbundance")].to("dimensionless").ndarray_view()
Z   = scg[("gas","Metallicity")].to("dimensionless").ndarray_view()

# 6. Compute hydrogen number density and n_HI
X   = 0.76
m_p = 1.6726219e-24
n_H  = rho * X / m_p
n_HI = n_H * xHI

# 7. Cell sizes (cm)
Lbox = (right - left).to("cm").value
dx, dy, dz = Lbox / N

# 8. Save everything to .npz
np.savez(outfile,
         n_HI=n_HI, T=T, vx=vx, vy=vy, vz=vz, Z=Z,
         dx=dx, dy=dy, dz=dz,
         left=left.to("cm").value, Lbox=Lbox)

print(f"[OK] Wrote {outfile}")
print("Grid shape:", n_HI.shape, "Box size [cm]:", Lbox)
print("n_HI min/max:", n_HI.min(), n_HI.max())
print("T median [K]:", np.nanmedian(T))


ImportError: cannot import name 'add_smoothed_particle_field' from 'yt.fields.api' (/user1/supriyo/miniconda3/envs/rascas-env/lib/python3.9/site-packages/yt/fields/api.py)

In [10]:
import numpy as np
import yt

def gadget_to_grid(snapfile, outfile="grid_rtfields.npz", N=64):
    """
    Convert Gadget HDF5 snapshot into a uniform grid for RASCAS.

    Parameters
    ----------
    snapfile : str
        Path to Gadget snapshot file (HDF5).
    outfile : str
        Name of output npz file with grid fields.
    N : int
        Grid resolution (N^3).
    """

    # 1. Load snapshot
    ds = yt.load(snapfile)

    # 2. Get gas particles (PartType0)
    ad = ds.all_data()
    coords = ad[("PartType0","Coordinates")].to("cm").ndarray_view()
    masses = ad[("PartType0","Masses")].to("g").ndarray_view()
    rho    = ad[("PartType0","Density")].to("g/cm**3").ndarray_view()
    temp   = ad[("PartType0","Temperature")].to("K").ndarray_view()
    vel    = ad[("PartType0","Velocities")].to("cm/s").ndarray_view()
    xHI    = ad[("PartType0","NeutralHydrogenAbundance")].ndarray_view()
    Z      = ad[("PartType0","metallicity")].ndarray_view()

    # 3. Define target grid
    left  = ds.domain_left_edge.to("cm").value
    right = ds.domain_right_edge.to("cm").value
    Lbox  = right - left
    dx, dy, dz = Lbox / N
    dims = (N, N, N)

    # 4. Deposit particles into grid (nearest-cell assignment)
    n_HI = np.zeros(dims)
    T    = np.zeros(dims)
    vx   = np.zeros(dims); vy = np.zeros(dims); vz = np.zeros(dims)
    Zg   = np.zeros(dims)
    count = np.zeros(dims)

    X   = 0.76
    m_p = 1.6726219e-24

    for i in range(coords.shape[0]):
        x, y, z = coords[i] - left
        ix = min(N-1, max(0, int(x/dx)))
        iy = min(N-1, max(0, int(y/dy)))
        iz = min(N-1, max(0, int(z/dz)))

        n_HI[ix,iy,iz] += (rho[i]*X/m_p) * xHI[i]
        T[ix,iy,iz]    += temp[i]
        vx[ix,iy,iz]   += vel[i,0]
        vy[ix,iy,iz]   += vel[i,1]
        vz[ix,iy,iz]   += vel[i,2]
        Zg[ix,iy,iz]   += Z[i]
        count[ix,iy,iz] += 1

    # 5. Average over particles per cell
    mask = count > 0
    T[mask]  /= count[mask]
    vx[mask] /= count[mask]
    vy[mask] /= count[mask]
    vz[mask] /= count[mask]
    Zg[mask] /= count[mask]

    # 6. Save to .npz
    np.savez(outfile,
             n_HI=n_HI, T=T, vx=vx, vy=vy, vz=vz, Z=Zg,
             dx=dx, dy=dy, dz=dz,
             left=left, Lbox=Lbox)

    print(f"[OK] Wrote {outfile}")
    print("Grid shape:", n_HI.shape)
    print("n_HI nonzero fraction:", np.count_nonzero(n_HI)/n_HI.size)
    print("T median [K]:", np.nanmedian(T))


# --- Example usage in notebook ---
gadget_to_grid("snapshot_200.hdf5", "grid_rtfields.npz", N=64)


yt : [INFO     ] 2025-09-17 21:24:36,568 ComovingIntegrationOn does not exist, falling back to OmegaLambda
yt : [INFO     ] 2025-09-17 21:24:36,644 Calculating time from 3.448e-01 to be 1.108e+17 seconds
yt : [INFO     ] 2025-09-17 21:24:36,646 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2025-09-17 21:24:36,694 Parameters: current_time              = 1.1075810732534834e+17 s
yt : [INFO     ] 2025-09-17 21:24:36,694 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-09-17 21:24:36,695 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-09-17 21:24:36,695 Parameters: domain_right_edge         = [64000. 64000. 64000.]
yt : [INFO     ] 2025-09-17 21:24:36,696 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-09-17 21:24:36,696 Parameters: current_redshift          = 1.8999965286929705
yt : [INFO     ] 2025-09-17 21:24:36,697 Parameters: omega_lambda              = 0.721
yt : [INFO     ] 2025-09-17 21:24:36,697 Para

[OK] Wrote grid_rtfields.npz
Grid shape: (64, 64, 64)
n_HI nonzero fraction: 0.003566741943359375
T median [K]: 0.0


In [5]:
# save as prep_gadget_to_rascas.py  (Python ≥3.9)
import numpy as np
import pynbody as pb
from pathlib import Path

SNAP = "snapshot_200.hdf5"      # << your Gadget HDF5
N     = 256                     # grid resolution (try 128–256 first)
WIDTH = '50 kpc'                # box width you want to cover

outdir = Path("rascas_grid")
outdir.mkdir(exist_ok=True)

print("Loading snapshot…")
s = pb.load(SNAP)
s.physical_units()      # convert to physical units (kpc, km/s, K, etc.)

g = s.g                 # gas only

# === Render 3D grids by SPH interpolation ===
print("Smoothing to a 3D grid…")
# ...existing code...
center = g['pos'].mean(axis=0)  # or use s['pos'].mean(axis=0) for all particles
boxsize = float(pb.units.Unit(WIDTH).in_units('kpc'))

# ...existing code...
center = g['pos'].mean(axis=0)  # or use s['pos'].mean(axis=0) for all particles
boxsize = float(pb.units.Unit(WIDTH).in_units('kpc'))

# ...existing code...
rho  = pb.sph.map3d_array(g, 'rho', N, N, N, 
                          x2=boxsize, y2=boxsize, z2=boxsize, 
                          xcen=center[0], ycen=center[1], zcen=center[2])
temp = pb.sph.map3d_array(g, 'temp', N, N, N, 
                          x2=boxsize, y2=boxsize, z2=boxsize, 
                          xcen=center[0], ycen=center[1], zcen=center[2])
vx   = pb.sph.map3d_array(g, 'vx', N, N, N, 
                          x2=boxsize, y2=boxsize, z2=boxsize, 
                          xcen=center[0], ycen=center[1], zcen=center[2])
vy   = pb.sph.map3d_array(g, 'vy', N, N, N, 
                          x2=boxsize, y2=boxsize, z2=boxsize, 
                          xcen=center[0], ycen=center[1], zcen=center[2])
vz   = pb.sph.map3d_array(g, 'vz', N, N, N, 
                          x2=boxsize, y2=boxsize, z2=boxsize, 
                          xcen=center[0], ycen=center[1], zcen=center[2])
# ...existing code...

# For metals and HI:
if 'metals' in g.loadable_keys():
    Z = pb.sph.map3d_array(g, 'metals', N, N, N, 
                           x2=boxsize, y2=boxsize, z2=boxsize, 
                           xcen=center[0], ycen=center[1], zcen=center[2])
else:
    Z = np.zeros_like(temp) + 0.002

xHI = None
for key in ('HI', 'NeutralHydrogenFraction', 'xHI'):
    if key in g.keys():
        xHI = pb.sph.map3d_array(g, key, N, N, N, 
                                 x2=boxsize, y2=boxsize, z2=boxsize, 
                                 xcen=center[0], ycen=center[1], zcen=center[2])
        break
if xHI is None:
    xHI = np.ones_like(temp)
# ...existing code...
# Optional metallicity (mass fraction) if present:
if 'metals' in g.loadable_keys():
    Z = pb.sph.render_3d_grid(g, quantity=g['metals'], nx=N, ny=N, nz=N, width=WIDTH)
else:
    Z = np.zeros_like(temp) + 0.002  # ~10% solar fallback

# Optional neutral fraction; fall back to fully neutral if not present:
xHI = None
for key in ('HI', 'NeutralHydrogenFraction', 'xHI'):
    if key in g.keys():
        xHI = pb.sph.render_3d_grid(g, quantity=g[key], nx=N, ny=N, nz=N, width=WIDTH)
        break
if xHI is None:
    xHI = np.ones_like(temp)   # assume neutral (you can replace with your own xHI model)

# Convert rho to number density of H and then to n_HI:
# rho [M_sun/kpc^3] -> g/cm^3, then n_H = X_H * rho / m_p
Msun = 1.98847e33
kpc  = 3.085677581e21
mp   = 1.6726219e-24
XH   = 0.76                      # hydrogen mass fraction
rho_cgs = rho * Msun / (kpc**3)  # g/cm^3
nH = XH * rho_cgs / mp           # cm^-3
nHI = nH * xHI                   # cm^-3

# Build grid coordinates (cell centers), normalized to [0,1]
# pynbody renders a cubic box of WIDTH; we’ll define edges from 0..1
xc = (np.arange(N) + 0.5) / N
yc = (np.arange(N) + 0.5) / N
zc = (np.arange(N) + 0.5) / N
dx_norm = 1.0 / N

# Save mesh geometry
np.savez(outdir / "grid_mesh.npz",
         xc=xc, yc=yc, zc=zc, dx=dx_norm,
         box_width_kpc=float(pb.units.Unit(WIDTH).in_units('kpc')),
         note="centers normalized to [0,1] in each axis")

# Save radiative transfer fields (names chosen to match RASCAS overwrite mode)
np.savez(outdir / "grid_rtfields.npz",
         nHI=nHI.astype(np.float32),     # cm^-3
         T=temp.astype(np.float32),      # K
         vx=vx.astype(np.float32),       # km/s
         vy=vy.astype(np.float32),       # km/s
         vz=vz.astype(np.float32),       # km/s
         Z=Z.astype(np.float32))         # mass fraction
print("Wrote rascas_grid/grid_mesh.npz and rascas_grid/grid_rtfields.npz")


Loading snapshot…
Smoothing to a 3D grid…


AttributeError: module 'pynbody.sph' has no attribute 'map3d_array'

In [5]:
import yt
import numpy as np
from astropy import constants as c, units as u

# ------------------------------
# Input snapshot
# ------------------------------
snap = "snapshot_200.hdf5"   # change to your file
ds   = yt.load(snap)
ad   = ds.all_data()

# ------------------------------
# Center = densest gas particle
# ------------------------------
dens   = ad[("gas","density")]                     # auto-derived by yt
coords = ad[("PartType0","Coordinates")].to("cm")  # gas particle positions

imax   = np.argmax(dens)
center = coords[imax].value                        # (3,) array in cm

# ------------------------------
# Define box size around center
# ------------------------------
box_kpc = 40.0/(1+ds.current_redshift)             # side length, physical (kpc)
L = (box_kpc * u.kpc).to(u.cm).value               # convert to cm

N = 256                                            # grid resolution
left  = center - 0.5*L
right = center + 0.5*L

# ------------------------------
# Interpolate fields on uniform grid
# ------------------------------
cg = ds.arbitrary_grid(left_edge=left,
                       right_edge=right,
                       dims=(N, N, N))

rho = cg[("gas","density")].to_value("g/cm**3")     # density
T   = cg[("gas","temperature")].to_value("K")       # temperature
Z   = cg[("gas","metallicity")]                     # metallicity (dimensionless)
vx  = cg[("gas","velocity_x")].to_value("cm/s")
vy  = cg[("gas","velocity_y")].to_value("cm/s")
vz  = cg[("gas","velocity_z")].to_value("cm/s")

# ------------------------------
# Convert to RASCAS units
# ------------------------------
m_p = (1.0 * c.m_p).cgs.value
mu  = 1.3                                # mean molecular weight
nH  = rho / (mu * m_p)                   # hydrogen number density [cm^-3]

# If neutral fraction available, use it. Else assume fully neutral:
xHI = np.ones_like(nH)
nHI = nH * xHI

vx_kms, vy_kms, vz_kms = vx/1e5, vy/1e5, vz/1e5     # convert to km/s

# ------------------------------
# Save NPZ file
# ------------------------------
np.savez("grid_rtfields.npz",
         nHI=nHI.astype("f4"),
         T=T.astype("f4"),
         vx=vx_kms.astype("f4"),
         vy=vy_kms.astype("f4"),
         vz=vz_kms.astype("f4"),
         Z=Z.astype("f4"),
         left=left.astype("f8"),
         right=right.astype("f8"),
         N=np.array([N], dtype="i4"))

print("✅ Wrote grid_rtfields.npz")


yt : [INFO     ] 2025-09-19 00:52:31,694 ComovingIntegrationOn does not exist, falling back to OmegaLambda
yt : [INFO     ] 2025-09-19 00:52:31,830 Calculating time from 3.448e-01 to be 1.108e+17 seconds
yt : [INFO     ] 2025-09-19 00:52:31,831 Assuming length units are in kpc/h (comoving)
yt : [INFO     ] 2025-09-19 00:52:31,873 Parameters: current_time              = 1.1075810732534834e+17 s
yt : [INFO     ] 2025-09-19 00:52:31,873 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-09-19 00:52:31,874 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-09-19 00:52:31,874 Parameters: domain_right_edge         = [64000. 64000. 64000.]
yt : [INFO     ] 2025-09-19 00:52:31,874 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-09-19 00:52:31,874 Parameters: current_redshift          = 1.8999965286929705
yt : [INFO     ] 2025-09-19 00:52:31,875 Parameters: omega_lambda              = 0.721
yt : [INFO     ] 2025-09-19 00:52:31,875 Para

✅ Wrote grid_rtfields.npz
